#### 가중치 데이터 다운로드받기(모델은 keras 내장)

In [1]:
import requests

url = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
r = requests.get(url, allow_redirects=True)

open('inception_v3.h5', 'wb').write(r.content)

87910968

#### 이미지 전처리하기

In [14]:
import tensorflow as tf

train_ds = tf.keras.preprocessing.image_dataset_from_directory('../CatDog/dataset/',
                                                               image_size=(150,150),
                                                               batch_size=64, # 이미지 2만장 한번에 안 넣고 64개씩 넣기
                                                               subset='training', 
                                                               validation_split=0.2, # 윗줄이랑 같이 써서 해당 비율로 쪼개짐 validation할 거 몇개 뽑아놓기
                                                               seed=12345, # 랜덤 시드 이것도 써야 오류안남
                                                              )

val_ds = tf.keras.preprocessing.image_dataset_from_directory('../CatDog/dataset/',
                                                               image_size=(150,150),
                                                               batch_size=64, # 이미지 2만장 한번에 안 넣고 64개씩 넣기
                                                               subset='validation', 
                                                               validation_split=0.2, # 안쓰면 오류나더라 어차피 validatio이 20%됨.
                                                               seed=12345,
                                                              ) 
### 이미지라면 0~255를 0~1로 압축하면 연산 속도 및 정확도가 높아짐
def prepro(i,y):
    i = tf.cast(i/255.0, tf.float32)
    return i, y

train_ds = train_ds.map(prepro)
val_ds = val_ds.map(prepro)


# import matplotlib.pyplot as plt  
# 사진 미리보기
# for i, y in train_ds.take(1):
#     plt.imshow( i[50].numpy().astype('uint8'))
#     plt.colorbar()
#     plt.show()
#     print(i[50], y)

# train_ds와 val_ds는 학습 데이터만 다른 동일한 형식인데, (학습데이터, 정답값) 이런 튜플이 담겨져 있다.
# cat폴더와 dog폴더로 나눠놓았었는데 알아서 2 classes라고 분류를 해놓는다.

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


#### 모델 사용하기

In [16]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers, Model # 이따 쓸거임 이거 안 하면 쓸 때마다 tf.tensorflow.keras.layers 해야함

inception_model = InceptionV3( input_shape=(150,150,3), include_top=False) 
# 맘대로 input_shape 바꿈 (150x150의 컬러사진)근데 이건 위의 이미지전처리 size랑 같게해줘야함
# include_top은 top인데 맨마지막 출력층 의미함
# 원래 출력은 1000개 물건 구분이고 내가 할 건 개고양이 구분이니 출력레이어 알아서 만들 거임
inception_model.load_weights('./inception_v3.h5') # 가중치 로드
inception_model.summary()

for i in inception_model.layers:
    i.trainable = False # 사용할 레이어의 w값은 학습시키면 안되므로 학습불가설정

    
# 이렇게 하면 특정 이름을 가진 레이어부터는 학습 가능하게 됨
unfreeze = False
for i in inception_model.layers:
    if i.name == 'mixed6':
        unfreeze == True
    if unfreeze == True:
        i.trainable  = True
    

last_layer = inception_model.get_layer('mixed7') # 중간에 해당 이름 가진 레이어 자르기

# 직접 레이어 만들고 연결하기
layer1 = tf.keras.layers.Flatten()(last_layer.output) #레이어가 아닌 레이어의 output을 써줘야 한다
layer2 = tf.keras.layers.Dense(1024, activation='relu')(layer1)
drop1 = tf.keras.layers.Dropout(0.2)(layer2)
layer3 = tf.keras.layers.Dense(1, activation='sigmoid')(drop1)

# 최종 모델 만들기
model = tf.keras.Model(inception_model.input, layer3) #input, output 레이어 써줘야함

model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.00001),metrics=['acc']), # acc도 되나봄
model.fit(train_ds, validation_data=val_ds, epochs=2)

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_564 (Conv2D)            (None, 74, 74, 32)   864         ['input_7[0][0]']                
                                                                                                  
 batch_normalization_564 (Batch  (None, 74, 74, 32)  96          ['conv2d_564[0][0]']             
 Normalization)                                                                                   
                                                                                       

 Normalization)                                                                                   
                                                                                                  
 batch_normalization_574 (Batch  (None, 16, 16, 96)  288         ['conv2d_574[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_575 (Batch  (None, 16, 16, 32)  96          ['conv2d_575[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_569 (Activation)    (None, 16, 16, 64)   0           ['batch_normalization_569[0][0]']
                                                                                                  
 activatio

 activation_586 (Activation)    (None, 16, 16, 64)   0           ['batch_normalization_586[0][0]']
                                                                                                  
 conv2d_584 (Conv2D)            (None, 16, 16, 48)   13824       ['mixed1[0][0]']                 
                                                                                                  
 conv2d_587 (Conv2D)            (None, 16, 16, 96)   55296       ['activation_586[0][0]']         
                                                                                                  
 batch_normalization_584 (Batch  (None, 16, 16, 48)  144         ['conv2d_584[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_587 (Batch  (None, 16, 16, 96)  288         ['conv2d_587[0][0]']             
 Normaliza

 mixed3 (Concatenate)           (None, 7, 7, 768)    0           ['activation_590[0][0]',         
                                                                  'activation_593[0][0]',         
                                                                  'max_pooling2d_26[0][0]']       
                                                                                                  
 conv2d_598 (Conv2D)            (None, 7, 7, 128)    98304       ['mixed3[0][0]']                 
                                                                                                  
 batch_normalization_598 (Batch  (None, 7, 7, 128)   384         ['conv2d_598[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_598 (Activation)    (None, 7, 7, 128)    0           ['batch_normalization_598[0][0]']
          

                                                                                                  
 batch_normalization_608 (Batch  (None, 7, 7, 160)   480         ['conv2d_608[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_608 (Activation)    (None, 7, 7, 160)    0           ['batch_normalization_608[0][0]']
                                                                                                  
 conv2d_609 (Conv2D)            (None, 7, 7, 160)    179200      ['activation_608[0][0]']         
                                                                                                  
 batch_normalization_609 (Batch  (None, 7, 7, 160)   480         ['conv2d_609[0][0]']             
 Normalization)                                                                                   
          

...

 conv2d_638 (Conv2D)            (None, 7, 7, 192)    258048      ['activation_637[0][0]']         
                                                                                                  
 batch_normalization_634 (Batch  (None, 7, 7, 192)   576         ['conv2d_634[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_638 (Batch  (None, 7, 7, 192)   576         ['conv2d_638[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_634 (Activation)    (None, 7, 7, 192)    0           ['batch_normalization_634[0][0]']
                                                                                                  
 activatio

                                                                                                  
 activation_642 (Activation)    (None, 3, 3, 384)    0           ['batch_normalization_642[0][0]']
                                                                                                  
 activation_643 (Activation)    (None, 3, 3, 384)    0           ['batch_normalization_643[0][0]']
                                                                                                  
 activation_646 (Activation)    (None, 3, 3, 384)    0           ['batch_normalization_646[0][0]']
                                                                                                  
 activation_647 (Activation)    (None, 3, 3, 384)    0           ['batch_normalization_647[0][0]']
                                                                                                  
 batch_normalization_648 (Batch  (None, 3, 3, 192)   576         ['conv2d_648[0][0]']             
 Normaliza

                                                                                                  
 activation_656 (Activation)    (None, 3, 3, 384)    0           ['batch_normalization_656[0][0]']
                                                                                                  
 batch_normalization_657 (Batch  (None, 3, 3, 192)   576         ['conv2d_657[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_649 (Activation)    (None, 3, 3, 320)    0           ['batch_normalization_649[0][0]']
                                                                                                  
 mixed9_1 (Concatenate)         (None, 3, 3, 768)    0           ['activation_651[0][0]',         
                                                                  'activation_652[0][0]']         
          

/usr/local/lib/python3.8/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2023-04-09 10:43:59.742623: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 470 of 512
2023-04-09 10:44:00.497338: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.
2023-04-09 10:44:01.580418: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8204
2023-04-09 10:44:02.463880: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


313/313 [==============================] - 252s 748ms/step - loss: 0.1301 - acc: 0.9495 - val_loss: 0.0842 - val_acc: 0.9660
Epoch 2/2
313/313 [==============================] - 38s 118ms/step - loss: 0.0548 - acc: 0.9810 - val_loss: 0.0770 - val_acc: 0.9696


#### 모델 저장 / 로드하는 방법

In [16]:
model.save("./savedModels/model2") ## 모델 저장하기
model2 = tf.keras.models.load_model("./savedModels/model2") # 모델 불러오기

INFO:tensorflow:Assets written to: ./savedModels/model2/assets
